In [1]:
import numpy as np
from os.path import normpath as fn # Fix Linux/Windows path issue
import sys
sys.path.append("nn") # add the nn module into system path

import nn.layer as layer
import nn.graph as graph
import nn.solver as solver
import nn.container as container
import nn.loss as loss

np.random.seed(0)

In [2]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))
train_im = np.float32(data['im_train'])/255.-0.5
train_im = np.reshape(train_im,[-1,28,28,1])
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_im = np.reshape(val_im,[-1,28,28,1])
val_lb = data['lbl_val']

In [3]:
###################################
# build static computational graph
###################################
C1 = 8
C2 = 16

# placeholder for input
inp = layer.Tensor()
lab = layer.Tensor()

model = container.Sequential([layer.Conv2d(1, C1, kernel_size=3, stride=2, padding=1, bias=True),
                           layer.RELU(),
                           layer.Conv2d(C1, C2, kernel_size=3, stride=2, padding=1, bias=True),
                           layer.RELU(),
                        #    layers.Dropout(0.2),
                           layer.Flatten(),
                           layer.Linear(C2*49 , 10)
                          ])
print(model)

# loss function: softmax + crossentropy
smaxloss = loss.SmaxCELoss()
accuracy = loss.Accuracy()

# build computational graph
y = model(inp)

# Cross Entropy of Soft-max
criterion = smaxloss(y,lab)

# Accuracy
acc = accuracy(y,lab)

Sequential(
  (0) Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=2, padding=1, bias=True)
  (1) RELU()
  (2) Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1, bias=True)
  (3) RELU()
  (4) Flatten()
  (5) Linear(in_features=784, out_features=10)
)


In [4]:
lr=0.01
NUM_EPOCH=10
DISPITER=100
BSZ=64

optimizer = solver.Momentum(graph.params, lr, mom=0.9)

In [5]:
# Training loop
batches = range(0,len(train_lb)-BSZ+1,BSZ)
niter, avg_loss, avg_acc=0, 0., 0.

for ep in range(NUM_EPOCH+1):

    # As we train, let's keep track of val accuracy
    vacc, vloss, viter= 0., 0., 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        inp.set(val_im[b:b+BSZ,...])
        lab.set(val_lb[b:b+BSZ])
        graph.Forward()
        viter += 1
        vacc += acc.top
        vloss += criterion.top
    vloss, vacc = vloss / viter, vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break

    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],...])
        lab.set(train_lb[idx[b:b+BSZ]])

        graph.Forward()
        avg_loss += criterion.top 
        avg_acc += acc.top
        niter += 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER
            avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss, avg_acc = 0., 0.

        graph.Backward(criterion)
        optimizer.step()
        

000000000: #### 0 Epochs: Val Loss = 2.312e+00, Accuracy = 12.29%
000000100: Training Loss = 1.857e+00, Accuracy = 51.67%
000000200: Training Loss = 9.819e-01, Accuracy = 78.08%
000000300: Training Loss = 7.238e-01, Accuracy = 82.52%
000000390: #### 1 Epochs: Val Loss = 5.416e-01, Accuracy = 88.23%
000000400: Training Loss = 6.324e-01, Accuracy = 85.06%
000000500: Training Loss = 5.894e-01, Accuracy = 86.08%
000000600: Training Loss = 5.773e-01, Accuracy = 85.88%
000000700: Training Loss = 5.048e-01, Accuracy = 87.19%
000000780: #### 2 Epochs: Val Loss = 4.069e-01, Accuracy = 90.10%
000000800: Training Loss = 4.703e-01, Accuracy = 87.98%
000000900: Training Loss = 4.750e-01, Accuracy = 87.55%
000001000: Training Loss = 4.683e-01, Accuracy = 88.06%
000001100: Training Loss = 4.450e-01, Accuracy = 88.22%
000001170: #### 3 Epochs: Val Loss = 4.003e-01, Accuracy = 90.00%
000001200: Training Loss = 4.454e-01, Accuracy = 88.56%
000001300: Training Loss = 4.583e-01, Accuracy = 87.94%
00000140